In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

In [ ]:
import os 
os.listdir('../input')

# 1. Carregando os dados

## 1.1 Carregando dados de treino

In [ ]:
dataTrain = pd.read_csv('../input/train.csv')
print(dataTrain.shape)
dataTrain.head()

In [ ]:
XdataTrain = dataTrain.iloc[:, 1:-1]
YdataTrain = dataTrain['median_house_value']
IDdataTrain = dataTrain['Id']

## 1.2 Carregando dados de teste

In [ ]:
dataTest = pd.read_csv('../input/test.csv')
print(dataTest.shape)
XdataTest = dataTest.iloc[:, 1:]
IDdataTest = dataTest['Id']
dataTest.head()

# 2. Vizualizando os dados 

## 2.1 Média e desvio padrão

#### Média

In [ ]:
dataTrain.mean()[1:-1]

In [ ]:
plot = dataTrain.mean()[4:-1].plot('bar')

In [ ]:
plot = dataTrain.mean()[4:-2].plot('bar')

#### Std

In [ ]:
dataTrain.std()[1:-1]

In [ ]:
plot = dataTrain.std()[4:-1].plot('bar')

In [ ]:
plot = dataTrain.mean()[3:-2].plot('bar')

In [ ]:
plot = dataTrain.std()[4:-2].plot('bar')

## 2.2 Matriz de correlação 

In [ ]:
plot = plt.matshow(dataTrain.corr())

In [ ]:
plot = dataTrain.corr().iloc[1:-1, -1].plot('bar')

## 2.3 Distribuição dos dados dependendo dos feartures

In [ ]:
plot = dataTrain.iloc[:, 3:-1].plot(kind='hist', bins=100, legend=True, alpha=0.3, xlim=(-1000, 70000))

Não consigo vizualizar direito, selecionar cada porção de dados com média parecida

In [ ]:
plot = dataTrain.iloc[:, 3:6].plot(kind='hist', bins=100, legend=True, alpha=0.3, xlim=(-100, 6000))

In [ ]:
plot = dataTrain.iloc[:, 6:-1].plot(kind='hist', bins=100, legend=True, alpha=0.3, xlim=(-1000, 70000))

## 2.4 Distribuição dos dados dependendo dos feartures e da target

In [ ]:
import matplotlib.colors as mcolors
for i in list(XdataTrain):
    plt.hist2d(XdataTrain[i], YdataTrain, bins=100, norm=mcolors.PowerNorm(0.3))
    plt.title(i)
    plt.show()

## 2.5 Analisando a distribuição do Target

In [ ]:
plot = dataTrain['median_house_value'].plot(kind='hist', bins=100, alpha=0.5)

In [ ]:
dataTrain['median_house_value'].max()

In [ ]:
(dataTrain['median_house_value']==500001).sum()

Existe uma quantidade muito grande de imóveis com median_house_value = 500001, provavelmente todos os imóveis com valor acima de 500,000 dólares são tabelados como 500,001 dólares

In [ ]:
dataTrain['median_house_value'].min()

In [ ]:
(dataTrain['median_house_value']==14999).sum()

Parece que o mesmo ocorre com imóveis com valor menor a 15,000 dólares sendo tabelados como tendo 14,999 dólares

# 3. Tratando os dados

## 3.1 Balancenado os dados

O limite de U$50.000,00 np valor da casa causa um pico de observações com median_house_value=500001, para se balancear os dados é retirado uma porção das observações com median_house_value=500001

In [ ]:
XbalTrain = XdataTrain.copy()
YbalTrain = YdataTrain.copy()

In [ ]:
maxValor = (XdataTrain[(YdataTrain==500001)])
lenMax = len(XdataTrain[(YdataTrain==500001)])

np.random.seed(3508)

remove_n = int(4 * lenMax / 5)
drop_indices = np.random.choice(maxValor.index, remove_n, replace=False)

In [ ]:
XbalTrain = XbalTrain.drop(drop_indices)
YbalTrain = YbalTrain.drop(drop_indices)

XbalTrain = XbalTrain.dropna()
YbalTrain = YbalTrain.dropna()

print(XbalTrain.shape)
print(YbalTrain.shape)

In [ ]:
plot = YbalTrain.plot(kind='hist', bins=100, alpha=0.5)

## 3.2 Fazendo uma "polinômio de 2º Grau" com os dados

In [ ]:
X2grauTrain = XbalTrain.copy()

In [ ]:
headers = list(XdataTrain)
for i in headers:
    for j in headers:
        if i==j:
            X2grauTrain[i+'^2'] = X2grauTrain[i] * X2grauTrain[j]
            break
        else:
            X2grauTrain[i+'x'+j] = X2grauTrain[i] * X2grauTrain[j]

In [ ]:
X2grauTrain.head()

## 3.3 Fazendo uma "polinômio de 3º Grau" com os dados

In [ ]:
X3grauTrain = X2grauTrain.copy()

In [ ]:
for i in headers:
    for j in headers:
            if i==j:
                for k in headers:
                    if k==j:
                        X2grauTrain[i+'^3'] = X3grauTrain[i] * X3grauTrain[j] * X3grauTrain[k]
                        break
                    else:
                        X2grauTrain[i+'^2x'+k] = X3grauTrain[i] * X3grauTrain[j] * X3grauTrain[k]
                break
            else:
                for k in headers:
                    if k==j:
                        X2grauTrain[i+'x'+j+'^2'] = X3grauTrain[i] * X3grauTrain[j] * X3grauTrain[k]
                        break
                    else:
                        X2grauTrain[i+'x'+j+'x'+k] = X3grauTrain[i] * X3grauTrain[j] * X3grauTrain[k]

In [ ]:
X2grauTrain.head()

## 3.4 Apenas latitude e longitude, em 1º, 2º e 3º graus

#### 1º Grau

In [ ]:
Xpos1Train = XbalTrain.copy()

In [ ]:
Xpos1Train = Xpos1Train[['longitude', 'latitude']]

In [ ]:
Xpos1Train.head()

#### 2º Grau

In [ ]:
Xpos2Train = Xpos1Train.copy()

In [ ]:
headers = list(Xpos1Train)
for i in headers:
    for j in headers:
        if i==j:
            Xpos2Train[i+'^2'] = Xpos2Train[i] * Xpos2Train[j]
            break
        else:
            Xpos2Train[i+'x'+j] = Xpos2Train[i] * Xpos2Train[j]

In [ ]:
Xpos2Train.head()

#### 3º Grau

In [ ]:
Xpos3Train = Xpos2Train.copy()

In [ ]:
for i in headers:
    for j in headers:
            if i==j:
                for k in headers:
                    if k==j:
                        Xpos3Train[i+'^3'] = Xpos3Train[i] * Xpos3Train[j] * Xpos3Train[k]
                        break
                    else:
                        Xpos3Train[i+'^2x'+k] = Xpos3Train[i] * Xpos3Train[j] * Xpos3Train[k]
                break
            else:
                for k in headers:
                    if k==j:
                        Xpos3Train[i+'x'+j+'^2'] = Xpos3Train[i] * Xpos3Train[j] * Xpos3Train[k]
                        break
                    else:
                        Xpos3Train[i+'x'+j+'x'+k] = Xpos3Train[i] * Xpos3Train[j] * Xpos3Train[k]

In [ ]:
Xpos3Train.head()

## 3.5 Retirando latitude e longitude, em 1º, 2º e 3º graus

#### 1º Grau

In [ ]:
XnoPos1Train = XbalTrain.copy()

In [ ]:
XnoPos1Train = XnoPos1Train.iloc[:, 2:]

In [ ]:
XnoPos1Train.head()

#### 2º Grau

In [ ]:
XnoPos2Train = XnoPos1Train.copy()

In [ ]:
headers = list(XnoPos1Train)
for i in headers:
    for j in headers:
        if i==j:
            XnoPos2Train[i+'^2'] = XnoPos2Train[i] * XnoPos2Train[j]
            break
        else:
            XnoPos2Train[i+'x'+j] = XnoPos2Train[i] * XnoPos2Train[j]

In [ ]:
XnoPos2Train.head()

#### 3º Grau

In [ ]:
XnoPos3Train = XnoPos2Train.copy()

In [ ]:
for i in headers:
    for j in headers:
            if i==j:
                for k in headers:
                    if k==j:
                        XnoPos3Train[i+'^3'] = XnoPos3Train[i] * XnoPos3Train[j] * XnoPos3Train[k]
                        break
                    else:
                        XnoPos3Train[i+'^2x'+k] = XnoPos3Train[i] * XnoPos3Train[j] * XnoPos3Train[k]
                break
            else:
                for k in headers:
                    if k==j:
                        XnoPos3Train[i+'x'+j+'^2'] = XnoPos3Train[i] * XnoPos3Train[j] * XnoPos3Train[k]
                        break
                    else:
                        XnoPos3Train[i+'x'+j+'x'+k] = XnoPos3Train[i] * XnoPos3Train[j] * XnoPos3Train[k]

In [ ]:
XnoPos3Train.head()

# 4. Criando um scorer

In [ ]:
from sklearn.metrics import make_scorer

## 4.1 Aplicando ReLu nos dados

In [ ]:
def RMSLErelu(Y, Ypred):
    n = len(Y)
    soma = 0
    Y = np.array(Y)
    for i in range(len(Y)):
        #Caso Ypred[i] for negativo eu trato como se ele fosse 0
        if Ypred[i] > 0:
            soma += ( math.log(Ypred[i]+1) - math.log(Y[i]+1) )**2
        else:
            soma += math.log(Y[i]+1)**2
    return math.sqrt(soma / n)
scorerRelu = make_scorer(RMSLErelu)

## 4.2 Aplicando abs nos dados

In [ ]:
def RMSLEabs(Y, Ypred):
    n = len(Y)
    soma = 0
    Y = np.array(Y)
    for i in range(len(Y)):
        soma += ( math.log( abs(Ypred[i]) + 1 ) - math.log( Y[i] + 1 ) )**2
    return math.sqrt(soma / n)
scorerAbs = make_scorer(RMSLEabs)

# 5. Predições

## 5.1 Regressão Linear

### 5.1.1 Míminos erros quadrados

In [ ]:
from sklearn.linear_model import LinearRegression

#### ReLu

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, XbalTrain,YbalTrain, cv=10, scoring=scorerRelu)
print(scores)
print(scores.mean())

#### abs

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

### 5.1.2 LASSO

In [ ]:
from sklearn.linear_model import Lasso

#### ReLu

In [ ]:
clf = Lasso()
print(clf)
scores = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu)
print(scores)
print(scores.mean())

#### abs

In [ ]:
clf = Lasso()
print(clf)
scores = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

### 5.1.3 RIDGE

In [ ]:
from sklearn.linear_model import Ridge

#### ReLu

In [ ]:
clf = Ridge()
print(clf)
scores = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu)
print(scores)
print(scores.mean())

#### abs

In [ ]:
clf = Ridge()
print(clf)
scores = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

### 5.1.4 Conclusão

O método utilizado para encontrar os parametros tem pouca variação no score final, com apenas o método de mínimos quadrados sendo usado a partir de agora

### 5.1.5 Regressão Linear com dados 'em 2º Grau'

#### ReLu

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, X2grauTrain,YbalTrain, cv=10, scoring=scorerRelu)
print(scores)
print(scores.mean())

#### abs

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, X2grauTrain,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

In [ ]:
from sklearn.linear_model import LogisticRegression

### 5.1.6 Regressão Linear com dados 'em 3º Grau'

#### ReLu

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, X3grauTrain,YbalTrain, cv=10, scoring=scorerRelu)
print(scores)
print(scores.mean())

#### abs

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, X3grauTrain,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

### Obs: Intuitivamente, aplicar abs ao invés de ReLu trará melhores resultados

Chutar qualquer valor de um imóvel é melhor do que falar que o seu valor é zero, logo, a partir de agora será usado apenas o método de abs para se calcular o score de treino dos modelos.

No final, caso necessário, será aplicado abs nos valores de YdataPred

### 5.1.7 Regressão Linear com apenas latitude e longitude, em 1º, 2º e 3º graus

#### 1º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, Xpos1Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

#### 2º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, Xpos2Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

#### 3º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, Xpos3Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

### 5.1.8 Regressão Linear retirando latitude e longitude, em 1º, 2º e 3º graus

#### 1º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, XnoPos1Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

#### 2º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, XnoPos2Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

#### 3º Grau

In [ ]:
clf = LinearRegression()
print(clf)
scores = cross_val_score(clf, XnoPos3Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

## 5.2 KNN Regressor

### 5.2.1 Overview de como o score se comporta sobre diferentes K's e P's

Fazendo as predições

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from tqdm import tqdm

accuracies = {}

Ptam = 5
Ktam = 25

accZ = np.empty([Ptam, Ktam])*0

for k in tqdm(range(1,Ktam+1)):
    for p in range(1,Ptam+1):
        knn = KNeighborsRegressor(n_neighbors=k, p=p)
        scores = cross_val_score(knn, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
        
        accuracies[(k,p)] = scores.mean()
        accZ[p-1][k-1] = scores.mean()

Observando os valores com score máximo

In [ ]:
accuraciesSorted = list(accuracies.items())
accuraciesSorted.sort(key=lambda x: x[1])

accuraciesSorted[:10]

Plotando as acuraciso dependendo de K e P

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D, get_test_data

In [ ]:
fig = plt.figure(figsize=(20,5))
ax = fig.add_subplot(1, 3, 1, projection='3d')

accK = np.arange(1, Ktam+1)
accP = np.arange(1, Ptam+1)

accK, accP = np.meshgrid(accK, accP)

ax.plot_wireframe(accK, accP, accZ)

ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.plot_wireframe(accK, accP, accZ)
ax.view_init(30, 30)

ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.plot_wireframe(accK, accP, accZ)
ax.view_init(20,90)

Parece que o sistema valoriza valores de K altos e P=1

### 5.2.2 Tentando achar o melhor valor de K para P=1

In [ ]:
accuracies = {}

for k in tqdm(range(1,100)):
    knn = KNeighborsRegressor(n_neighbors=k, p=1)
    scores = cross_val_score(knn, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
    accuracies[k] = scores.mean()

In [ ]:
accuraciesSorted = list(accuracies.items())
accuraciesSorted.sort(key=lambda x: x[1])

accuraciesSorted[:10]

In [ ]:
accX = sorted(list(accuracies.keys()))
accY = [accuracies[i] for i in accX]
plot = plt.plot(accX, accY)

O valor ótimo parece estar entre 5 e 30

In [ ]:
plot = plt.plot(accX[4:30], accY[4:30], 'bo', accX[4:30], accY[4:30])

Posso aproximar uma linha para os pontos antre 6 10 e outra linha para os pontos entre 10 e 30 e pegar o K do ponto de intersecção dessas linhas

In [ ]:
Esq = np.polyfit(accX[4:10], accY[4:10], 1)
Dir = np.polyfit(accX[10:30], accY[10:30], 1)
print(round((Esq[1]-Dir[1])/(Dir[0]-Esq[0])))

### 5.2.3 Parece que os valoes ótimos de K e P para o modelo são 10 e 1

Fazendo a predição

In [ ]:
knn = KNeighborsRegressor(n_neighbors=10, p=1)
print(knn)
scores = cross_val_score(knn, XbalTrain, YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

## 5.3 KNN Regressor só com latitude o longitude e P=2

A ideia vem de tentar achar a distância real entre os imóveis

### 5.3.1 Achando o valor ótimo de K

In [ ]:
accuracies = {}

for k in tqdm(range(1,100)):
    knn = KNeighborsRegressor(n_neighbors=k, p=2)
    scores = cross_val_score(knn, Xpos1Train,YbalTrain, cv=10, scoring=scorerAbs)
    accuracies[k] = scores.mean()

In [ ]:
accuraciesSorted = list(accuracies.items())
accuraciesSorted.sort(key=lambda x: x[1])

accuraciesSorted[:10]

In [ ]:
accX = sorted(list(accuracies.keys()))
accY = [accuracies[i] for i in accX]
plot = plt.plot(accX, accY)

In [ ]:
plot = plt.plot(accX[3:10], accY[3:10], 'bo', accX[3:10], accY[3:10])

O valor ótimo de K no caso é claramente 6

### 5.3.2 Parece que os valoes ótimos de K para o modelo é 6

Fazendo a predição

In [ ]:
knn = KNeighborsRegressor(n_neighbors=6, p=2)
print(knn)
scores = cross_val_score(knn, Xpos1Train,YbalTrain, cv=10, scoring=scorerAbs)
print(scores)
print(scores.mean())

## 5.4 Árvores de Decisção

### 5.4.1 Uma única árvode de regressão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#### 5.4.1.1 Como um colocar um limite na profundidade altera o score?

In [ ]:
scores = {}
for i in tqdm(range(1,26)):
    clf = DecisionTreeRegressor(max_depth=i, random_state=3508)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()

clf = DecisionTreeRegressor(random_state=3508)
clf.fit(XdataTrain, YdataTrain)
scores[i+1] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()

In [ ]:
scrX = list(scores.keys())
scrY = list(scores.values())

plot = plt.plot(scrX, scrY, 'bo', scrX, scrY)

O score começa alto e diminuindo, depois volta a aumentar, se estabiliando por volta 0.340

In [ ]:
plot = plt.plot(scrX[7:13], scrY[7:13], 'bo', scrX[7:13], scrY[7:13])

Para os parâmetros inicias, o melhor valor para a profundidade máxima é claramente 10

#### 5.4.1.2 Como aumentar o número mínimo de observações num nó necessária para ocorrer um split altera o score?

In [ ]:
scores = {}
for i in tqdm(range(2,102)):
    clf = DecisionTreeRegressor(max_depth=10, min_samples_split=i, random_state=3508)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()

In [ ]:
scrX = list(scores.keys())
scrY = list(scores.values())

plot = plt.plot(scrX, scrY, 'bo', scrX, scrY)

In [ ]:
plot = plt.plot(scrX[10:90], scrY[10:90], 'bo', scrX[10:90], scrY[10:90])

A mudança no min_samples_split não altera muito o score, com o valor ótimo perto de 50

#### 5.4.1.3 Como aumentar o número mínimo de observações necessárias num nó para ele virar um nó folha altera o score?

In [ ]:
scores = {}
for i in tqdm(range(1,51)):
    clf = DecisionTreeRegressor(max_depth=10, min_samples_split=50, min_samples_leaf=i, random_state=3508)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()

In [ ]:
scrX = list(scores.keys())
scrY = list(scores.values())

plot = plt.plot(scrX, scrY, 'bo', scrX, scrY)

In [ ]:
plot = plt.plot(scrX[5:30], scrY[5:30], 'bo', scrX[5:30], scrY[5:30])

A mudança no min_samples_leaf não altera muito o score, com o valor ótimo perto de 15

#### 5.4.1.3 As alterações feitas modificaram o limite ótimo para a profundidade da árvore?

In [ ]:
scores = {}
for i in tqdm(range(1,51)):
    clf = DecisionTreeRegressor(max_depth=i, min_samples_split=50, min_samples_leaf=15, random_state=3508)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()

clf = DecisionTreeRegressor(min_samples_split=50, min_samples_leaf=15, random_state=3508)
scores[i+1] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
    
scrX = list(scores.keys())
scrY = list(scores.values())

plot = plt.plot(scrX, scrY, 'bo', scrX, scrY)
plt.show()
plot = plt.plot(scrX[7:13], scrY[7:13], 'bo', scrX[7:13], scrY[7:13])
plt.show()

Agora o score esbiliza no valor mínimo, com o fato de colocar um limine na profundidade da árvore não é uma alteração necessário ao modelo

### 5.4.2 Bagging Trees

In [ ]:
from sklearn.ensemble import BaggingRegressor

#### 5.4.2.1 Análise de como a quantidade de regressores modifica o score, com uma delta igual a 10

In [ ]:
import time

O score diminui rapidamente quando se altera os valores da quantidae n de árvores para valores desse n pequenos, apos isso a score continua a diminuir, porém muito mais devagar. O tempo necesário de treinamento cresce rapidamente linearmente com a quantidade de árvores, fazendo com que o score não vala a pena para o tempo necessário para o treinamento.

Esses gráficos foram criados com o código comentado, o código foi colocado para não rodar automaticamente pelo fato dele ser muito pesado. Apenas uma das iterações foi colocada para rodar como prova de que o código funciona.

![title](BaggQuantTree.png)

In [ ]:
'''scores = {}
temps = []
t = time.time()
for i in tqdm(range(1, 121, 10)):
    clf = BaggingRegressor(DecisionTreeRegressor(random_state=3508), random_state=1165842557, n_estimators=i)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
    temps.append(time.time()-t)
    t = time.time()

scrX = list(scores.keys())
scrY = list(scores.values())

plot = plt.plot(scrX, scrY, 'bo', scrX, scrY, scrX, temps)
plt.title('Score pela quantidade de árvores no Bagging')

plt.plot(scrX, temps, 'bo', scrX, temps)
plt.title('Tempo de treinamento pela quantidade \nde árvores no Bagging')'''
pass

In [ ]:
clf = BaggingRegressor(DecisionTreeRegressor(random_state=3508), random_state=1165842557, n_estimators=80)
print(clf)
score = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu)
print(score)
print(score.mean())

#### 5.4.2.2 Análise de como a profundidade máxima das árvores modifica o score

In [ ]:
scores = {}
temps = []
t = time.time()
for i in tqdm(range(1, 21)):
    clf = BaggingRegressor(DecisionTreeRegressor(max_depth=i, random_state=3508), random_state=1165842557, n_estimators=50)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
    temps.append(time.time()-t)
    t = time.time()
clf = BaggingRegressor(DecisionTreeRegressor(random_state=3508), random_state=1165842557, n_estimators=50)
scores[i+1] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
temps.append(time.time()-t)

scrX = list(scores.keys())
scrY = list(scores.values())

plt.plot(scrX, scrY, 'bo', scrX, scrY)
plt.show()
plt.plot(scrX, temps, 'bo', scrX, temps)
plt.show()

O score stabiliza perto da profundidade máxima perto de 20

### 5.4.3 Random Forrest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#### 5.4.3.1 Testando com as configurações básicas

In [ ]:
clf = RandomForestRegressor(n_estimators=100, max_features='sqrt', random_state=1165842557, )
print(clf)
score = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu)
print(score)
print(score.mean())

### 5.4.4 Gradient Tree Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#### 5.4.4.1 Acompanhando o comportamento dependendo da variação da quantidade de árvores, com a profundidade igual a 2

In [ ]:
scores = {}
temps = []
t = time.time()
for i in tqdm(range(50, 250, 10)):
    clf = GradientBoostingRegressor(max_depth=2, n_estimators=i)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
    temps.append(time.time()-t)
    t = time.time()
clf = GradientBoostingRegressor(max_depth=2)
scores[i+1] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
temps.append(time.time()-t)

scrX = list(scores.keys())
scrY = list(scores.values())

plt.plot(scrX, scrY, 'bo', scrX, scrY)
plt.show()
plt.plot(scrX, temps, 'bo', scrX, temps)
plt.show()

#### 5.4.4.2 Acompanhando o comportamento dependendo da profundidade das árvores, com 140 árvores

In [ ]:
scores = {}
temps = []
t = time.time()
for i in tqdm(range(1, 11)):
    clf = GradientBoostingRegressor(max_depth=i, n_estimators=140)
    scores[i] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
    temps.append(time.time()-t)
    t = time.time()
clf = GradientBoostingRegressor(n_estimators=140)
scores[i+1] = cross_val_score(clf, XbalTrain, YbalTrain, cv=10, scoring=scorerRelu).mean()
temps.append(time.time()-t)

scrX = list(scores.keys())
scrY = list(scores.values())

plt.plot(scrX, scrY, 'bo', scrX, scrY)
plt.show()
plt.plot(scrX, temps, 'bo', scrX, temps)
plt.show()

# 6. Predição dos dados de teste com o método de melhor score

In [ ]:
clf = GradientBoostingRegressor(max_depth=8, n_estimators=140)
clf.fit(Xpos1Train,YbalTrain)
YdataPred = clf.predict(XdataTest.iloc[:, :2])

In [ ]:
ID = list(IDdataTest)
YdataPred = list(YdataPred)

In [ ]:
submission = np.array([ID, YdataPred])

submission = pd.DataFrame(submission.T, columns=['Id', 'median_house_value'])
submission['Id'] = submission['Id'].astype(int)

In [ ]:
submission.to_csv('out.csv', index=False)